## SICP 习题 （2.57）解题总结：求导表达式里任意项参数的处理

SICP 习题 2.57 是 2.56的延伸，题目要求我们的求导函数不仅可以处理 `'(+ x y)` 这种形式，还可以处理 `(+ x y z)` 这中形式，而且，更宽泛地讲，需要我们处理`(+ x1 x2 x3 ...)`这样的形式。

这个解题方法在题干中有提示，我在2.56的题解中也有剧透，

基本做法就是把`(+ x1 x2 x3)`变成`(+ x1 (+ x2 x3 ...))`

这里面不用再担心`(+ x2 x3 ...)`里其它参数的处理，因为我们定义的deriv函数对递归调用，把`(+ x2 x3 ...)`当另一个加式进行处理，这样就可以处理任意项的参数了。


下面把两种方法都列出来：

加式的处理

In [1]:
;原来的方法
(define (augend-original s) (cddr s))

;新的方法
(define (augend s) 
   (if (null? (cdddr s)) 
       (caddr s) 
       (cons '+ (cddr s)))) 

乘式的处理

In [3]:
;原来的方法
(define (multiplicand-original p) (caddr p))

;新的方法
(define (multiplicand p) 
   (if (null? (cdddr p)) 
       (caddr p) 
       (cons '* (cddr p)))) 

下面把完整的代码列出来：

In [4]:
(define (variable? x) (symbol? x))

(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

(define (make-sum-original a1 a2) (list '+ a1 a2))

(define (make-sum a1 a2)
  (define (iter sum1 sum2)
    '())
  (cond ((=number? a1 0) a2)
	((=number? a2 0) a1)
	((and (number? a1) (number? a2)) (+ a1 a2))
	(else 
	 (list '+ a1 a2))))

(define (=number? exp num)
  (and (number? exp) (= exp num)))

(define (make-product-original m1 m2) (list '* m1 m2))

(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
	((=number? m1 1) m2)
	((=number? m2 1) m1)
	((and (number? m1) (number? m2)) (* m1 m2))
	(else (list '* m1 m2))))

(define (sum? x)
  (and (pair? x) (eq? (car x) '+)))

(define (addend s) (cadr s))

(define (augend-original s) (cddr s))
(define (augend s) 
   (if (null? (cdddr s)) 
       (caddr s) 
       (cons '+ (cddr s)))) 

(define (product? x)
  (and (pair? x) (eq? (car x) '*)))

(define (multiplier p) (cadr p))

(define (multiplicand-original p) (caddr p))


 (define (multiplicand p) 
   (if (null? (cdddr p)) 
       (caddr p) 
       (cons '* (cddr p)))) 


(define (make-exponentiation base exponent)
  (cond ((= exponent 0)
	 1)
	((= exponent 1)
	 base)
	(else
	 (list '** base exponent))))

(define (exponentiation? x)
  (and (pair? x)
       (eq? (car x) '**)))

(define (base exp)
  (cadr exp))

(define (exponent exp)
  (caddr exp))


(define (deriv exp var)
  (cond ((number? exp) 0)
	((variable? exp)
	 (if (same-variable? exp var) 1 0))
	((sum? exp)
	 (make-sum (deriv (addend exp) var)
		   (deriv (augend exp) var)))
	((product? exp)
	 (make-sum
	  (make-product (multiplier exp)
			(deriv (multiplicand exp) var))
	  (make-product (deriv (multiplier exp) var)
			(multiplicand exp))))
	((exponentiation? exp)
	 (let ((n (exponent exp))
	       (u (base exp)))
	   (make-product
	    n
	    (make-product
	     (make-exponentiation
	      u
	      (- n 1))
	     (deriv u var)))))
	(else 
	 (error "unkonwn expression type -- DERIV" exp))))


(define (start-test-2-57)
  (display (deriv '(+ x 3) 'x)) (newline)

  (display (deriv '(* x y) 'x)) (newline)

  (display (deriv '(* x y (+ x 3)) 'x))(newline)) 


In [5]:
(start-test-2-57)

1
y
(+ (* x y) (* y (+ x 3)))
